In [8]:
import tensorflow as tf
import numpy as np
from PIL import Image
import glob
import os
import matplotlib.pyplot as plt

print("TensorFlow:", tf.__version__)
print("NumPy:", np.__version__)
print("PIL import 완료!")


TensorFlow: 2.20.0
NumPy: 2.1.3
PIL import 완료!


In [9]:
base_path = r"C:\Users\kdh98\OneDrive\Desktop\모두의 연구소\7. DL"

# 1. resize (28x28) & data load

In [11]:
def resize_images(img_path):
    images = glob.glob(img_path + "/*.jpg")
    print(len(images), " images to be resized.")

    target_size = (28, 28)

    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.Resampling.LANCZOS)
        new_img.save(img, "JPEG")

    print(len(images), " images resized.")

def load_data(img_path):
    img_size = 28
    color = 3
    
    files_scissor = glob.glob(img_path + r"\scissor\*.jpg")
    files_rock = glob.glob(img_path + r"\rock\*.jpg")
    files_paper = glob.glob(img_path + r"\paper\*.jpg")

    all_files = files_scissor + files_rock + files_paper
    number_of_data = len(all_files)

    imgs = np.zeros((number_of_data, img_size, img_size, color), dtype=np.int32)
    labels = np.zeros(number_of_data, dtype=np.int32)

    idx = 0

    # 0 = scissor
    for file in files_scissor:
        imgs[idx] = np.array(Image.open(file), dtype=np.int32)
        labels[idx] = 0
        idx += 1

    # 1 = rock
    for file in files_rock:
        imgs[idx] = np.array(Image.open(file), dtype=np.int32)
        labels[idx] = 1
        idx += 1

    # 2 = paper
    for file in files_paper:
        imgs[idx] = np.array(Image.open(file), dtype=np.int32)
        labels[idx] = 2
        idx += 1

    print("총 이미지 개수:", idx)
    return imgs, labels

In [ ]:
# resize

In [12]:
resize_images(base_path + r"\scissor")
resize_images(base_path + r"\rock")
resize_images(base_path + r"\paper")

100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.


In [ ]:
# demension reset

In [13]:
x_all, y_all = load_data(base_path)
x_all = x_all / 255.0

print("x_all:", x_all.shape)
print("y_all:", y_all.shape)


총 이미지 개수: 300
x_all: (300, 28, 28, 3)
y_all: (300,)


# 2. split

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_all, y_all,
    test_size=0.2,
    shuffle=True,
    stratify=y_all,
    random_state=42
)


# 3. train & eval

In [17]:
from tensorflow import keras

n_channel_1 = 32
n_channel_2 = 64
n_dense = 64
n_train_epoch = 25

model = keras.models.Sequential([
    keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(n_dense, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

model.summary()


model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(x_train, y_train, epochs=n_train_epoch)





C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         102,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 122,051 (476.76 KB)

 Trainable params: 122,051 (476.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3333 - loss: 1.0907 
Epoch 2/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5458 - loss: 1.0483 
Epoch 3/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5792 - loss: 1.0136 
Epoch 4/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7458 - loss: 0.9443  
Epoch 5/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7458 - loss: 0.8728 
Epoch 6/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9250 - loss: 0.7306 
Epoch 7/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9333 - loss: 0.5648 
Epoch 8/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9417 - loss: 0.4452
Epoch 9/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9583 - loss: 0.3182
Epoch 10/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8667 - loss: 0.3908 
Epoch 11/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8375 - loss: 0.4365
Epoch 12/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8542 - loss: 0.295

In [18]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_accuracy:", test_accuracy)

2/2 - 0s - 90ms/step - accuracy: 1.0000 - loss: 0.0278
test_accuracy: 1.0
